In [1]:
import Parser
import Process_Data
import Feature_Engineering as fe

import zipfile
import datetime
import pandas as pd
import os

## DOWNLOAD DATA

In [2]:
## FOR MAC 
## Chrome version 90.0.4430.212
script_dir = os.path.dirname(os.path.realpath('chromedriver_mac'))
webdriver_path = os.path.join(script_dir, 'chromedriver_mac')

In [ ]:
## FOR WINDOWS
## Chrome version 91.0.4472.77
# script_dir = os.path.dirname(os.path.realpath('chromedriver.exe'))
# webdriver_path = os.path.join(script_dir, 'chromedriver.exe')
##

In [38]:
## UNPACK DATA
with zipfile.ZipFile("Data/data.zip", "r") as z:
    z.extractall("")

In [4]:
## PARSE
matches = []
hrefs = set()

startdate = datetime.date.today() + datetime.timedelta(1)
enddate = startdate + datetime.timedelta(2)
Parser.parse(startdate, enddate, matches, hrefs, webdriver_path)


## UPDATE OLD MATCHES
filename = 'Data/old_matches.csv'
old_matches = Process_Data.csv_to_list(filename)

matches = Process_Data.unpack_and_change(matches)
Process_Data.make_csv(matches, 'Data/new_matches.csv')
matches_after_parse = Process_Data.csv_to_list('Data/new_matches.csv')
Process_Data.make_csv(old_matches + matches_after_parse, 'Data/old_matches.csv')


## PROCESS OLD MATCHES
Process_Data.delete_trash(old_matches)
Process_Data.split_sets_and_dur(old_matches)
Process_Data.fix_set_score(old_matches)
Process_Data.split_stats(old_matches)
Process_Data.create_srv_games(old_matches)
Process_Data.create_bp_stat(old_matches)


## PROCESS NEW MATCHES
filename = 'Data/new_matches.csv'
new_matches = Process_Data.csv_to_list(filename)
Process_Data.delete_trash(new_matches)
Process_Data.split_sets_and_dur(new_matches)
Process_Data.fix_set_score(new_matches)
Process_Data.split_stats(new_matches)
Process_Data.create_srv_games(new_matches)
Process_Data.create_bp_stat(new_matches)

## UNION MATCHES --> CSV
matches = old_matches + new_matches
filename = 'Data/matches_final.csv'
Process_Data.to_csv(matches, filename)

## CSV --> PICKLE
data = pd.read_csv(filename)
filename = 'Data/matches_final.pkl'
data.to_pickle(filename)


Parsed matches on date: 2021-05-28 7


/Users/yasha_ten/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## FEATURE ENGINEERING

In [5]:
## READ DATA
df = pd.read_pickle('Data/matches_final.pkl') 
players1 = pd.read_pickle('Data/players.pkl')
players2 = pd.read_pickle('Data/players.pkl')

## CHANGE TIME TYPE
fe.fix_time(df)

## fix players' names for transporting stats
fe.fix_names1(players1)
fe.fix_names2(players2)
players1 = players1.set_index(['name'])
players2 = players2.set_index(['name'])

## transporting stats players --> dataframe
fe.add_stats(df, players1)
fe.add_stats(df, players2, 0)


## fix letters in data
fe.fix_letters(df)

## create retirement stat
fe.create_retirement_stat(df)


## converting age to datetime
fe.fix_age(df, 1)
fe.fix_age(df, 2)


## converting some stats to datetime in df for the next steps
df['date'] = pd.to_datetime(df['date'])
df['p1_birth'] = pd.to_datetime(df['p1_birth'])
df['p2_birth'] = pd.to_datetime(df['p2_birth'])

# count prc of winning points on return
fe.create_feature_WRP(df)

## normalize to per_game stats
fe.create_aces_per_game(df)
fe.create_double_faults_per_game(df)


## create stats based on common opponents
stats = ['first_serve_prc_match_player1', 'first_serve_points_prc_match_player1',
         'second_serve_points_prc_match_player1', 'winning_on_return_prc_player1',
        'aces_per_game_player1', 'df_per_game_player1', 'break_points_prc_match_player1']


fe.create_common_stats(df, stats)


## create new features
fe.create_feature_WSP(df)
fe.create_feature_COMPLETE(df)
fe.create_feature_SERVEADV(df)

# add invert matches before startdate
startdate = '2021-01-01'
df = fe.invert(df, startdate)

## drop Nan stats
df.dropna(subset=['k1'], inplace=True)
df.dropna(subset=['k2'], inplace=True)

# FIX SOME STATS' FORMAT
df = fe.fix_format(df)

# CREATE LAG FEATURES USING MAIN STATS
df = fe.build_lag_features(df)


df.rename(columns={"player1": "p1", "player2": "p2", "player1_win": "p1_win"}, inplace=True)
df = df.set_index(["date", "p1", "p2"])

df.head(3)

FIXING PLAYERS' NAMES...


IntProgress(value=0, max=1357)

IntProgress(value=0, max=1357)

PLAYERS' NAMES FIXED

IMPORTING STATIC STATS...


IntProgress(value=0, max=27623)

...

IntProgress(value=0, max=12657)

STAT first_serve_points_prc_match_common CREATED
ADDING INVERT MATCHES...
INVERT MATCHES ADDED

CREATING LAG FEATURES...


/Users/yasha_ten/anaconda3/lib/python3.7/site-packages/numpy/lib/function_base.py:3826: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


LAG FEATURES CREATED



index  \
date       p1          p2                   
2015-06-01 Chardy J.   Murray A.        0   
           Djokovic N. Gasquet R.       1   
           Gasquet R.  Djokovic N.      2   

                                                                            match_url  \
date       p1          p2                                                               
2015-06-01 Chardy J.   Murray A.      https://www.sofascore.com/chardy-murray/BcgsEqg   
           Djokovic N. Gasquet R.   https://www.sofascore.com/djokovic-gasquet/oNf...   
           Gasquet R.  Djokovic N.  https://www.sofascore.com/djokovic-gasquet/oNf...   

                                   status  Surface    k1     k2  p1_win  \
date       p1          p2                                                 
2015-06-01 Chardy J.   Murray A.    Ended      0.0   8.0   1.08       0   
           Djokovic N. Gasquet R.   Ended      0.0   1.0  19.00       1   
           Gasquet R.  Djokovic N.  Ended      0.0  19.0   1.00       0   

                                    round match_dur Location  sets_stat  \
date       p1          p2                                                 
2015-06-01 Chardy J.   Murray A.      1.0       173      NaN        NaN   
           Djokovic N. Gasquet R.     1.0       121      NaN        NaN   
           Gasquet R.  Djokovic N.    1.0       121      NaN        NaN   

                                    duration_set1  duration_set2  \
date       p1          p2                                          
2015-06-01 Chardy J.   Murray A.             39.0           50.0   
           Djokovic N. Gasquet R.            38.0           39.0   
           Gasquet R.  Djokovic N.           38.0           39.0   

                                    duration_set3  duration_set4  \
date       p1          p2                                          
2015-06-01 Chardy J.   Murray A.             41.0           41.0   
           Djokovic N. Gasquet R.            43.0            NaN   
           Gasquet R.  Djokovic N.           43.0            NaN   

                                    duration_set5  aces_match_player1  \
date       p1          p2                                               
2015-06-01 Chardy J.   Murray A.              NaN                 NaN   
           Djokovic N. Gasquet R.             NaN                 NaN   
           Gasquet R.  Djokovic N.            NaN                 NaN   

                                    aces_match_player2  \
date       p1          p2                                
2015-06-01 Chardy J.   Murray A.                   NaN   
           Djokovic N. Gasquet R.                  NaN   
           Gasquet R.  Djokovic N.                 NaN   

                                    double_faults_match_player1  \
date       p1          p2                                         
2015-06-01 Chardy J.   Murray A.                            NaN   
           Djokovic N. Gasquet R.                           NaN   
           Gasquet R.  Djokovic N.                          NaN   

                                    double_faults_match_player2  \
date       p1          p2                                         
2015-06-01 Chardy J.   Murray A.                            NaN   
           Djokovic N. Gasquet R.                           NaN   
           Gasquet R.  Djokovic N.                          NaN   

                                    first_serve_prc_match_player1  \
date       p1          p2                                           
2015-06-01 Chardy J.   Murray A.                              NaN   
           Djokovic N. Gasquet R.                             NaN   
           Gasquet R.  Djokovic N.                            NaN   

                                    first_serve_prc_match_player2  \
date       p1          p2                                           
2015-06-01 Chardy J.   Murray A.                              NaN   
           Djokovic N. Gasquet R.               

## CREATE MODELS

In [3]:
import Create_Models as cm

In [5]:
predictions = []

data = cm.create_data1(df)
y_pred1 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred1)
data = cm.create_data2(df)
y_pred2 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred2)
data = cm.create_data3(df)
y_pred3 = cm.build_model_classifier(data, startdate=startdate)
predictions.append(y_pred3)

## BUILD COMPOSITION

In [6]:
import Composition
agg = Composition.aggregate(df, startdate, predictions)

In [39]:
agg

,,p1_win_prob,p2_win_prob
p1,p2,,
Barrios Vera T.,Harrison C.,0.000000,0.000000
Escobar G.,Quiroz R.,0.425430,0.574570
Kozlov S.,Haerteis J.,0.597077,0.402923
Sakamoto P.,Young D.,0.360787,0.639213
Basic M.,Kuzmanov D.,0.493065,0.506935
...,...,...,...
Monfils G.,Nishioka Y.,0.653291,0.346709
Paul T.,Tsitsipas S.,0.204358,0.795642
Rinderknech A.,Sinner J.,0.176604,0.823396
